### MNISTデータセットから手書き数字判別モデルを作る

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
# ハイパーパラメータ
learning_rate = 0.001
batch_size = 64
epochs = 5

In [4]:
# データセット
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
training_data = datasets.MNIST(
    'data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(
    'data', train=False, transform=transform)

In [5]:
# データローダーのインスタンス
train_dataloader = DataLoader(training_data, batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size)

In [6]:
# デバイス
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

In [7]:
is_available = torch.cuda.is_available()
print(f"GPU利用可能: {is_available}")

if is_available:
    print(f"GPUデバイス名: {torch.cuda.get_device_name(0)}")
    print(f"現在のデバイス番号: {torch.cuda.current_device()}")

GPU利用可能: True
GPUデバイス名: NVIDIA GeForce RTX 4050 Laptop GPU
現在のデバイス番号: 0


In [ ]:
# ニューラルネットワークの定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)
        self.fc1 = nn.Linear(9216, 128)   # 9216は人力で求める
        # self.fc1 = nn.LazyLinear(128) # LazyLinearを使う場合
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        output = self.fc2(x)
        return output

In [9]:
# NNをインスタンス化してデバイスに転送
model = Net()
model.to(device)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)